# Ismael – Persoonlijke Notebook 
Dit notebook bevat de stappen die Ismael uitvoert volgens de planning.

## Overzicht taken
- **Dag 1:** Almere filteren, alleen wijken (WK), aantal wijken controleren
- **Dag 2:** Groei 15–25 per wijk analyseren
- **Dag 3:** Ratio jongeren / totaal bevolking maken
- **Dag 4:** MAE, RMSE, R² berekenen

> **Let op:** Kolomnamen kunnen bij jullie dataset nét anders heten.
Pas in dat geval de kolomnamen onderaan per sectie aan.

## Dataset Fixen
**Doel:** een dataset op wijkniveau (alleen Almere + alleen WK-codes).

### 1) Imports

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Matplotlib is building the font cache; this may take a moment.


### 2) Dataset inladen (output van Persoon 1)
Persoon 1 leest de ruwe CSV correct in en fixt de header.
Gebruik hier hetzelfde pad/bestandsnaam als jullie afspreken.

**Voorbeeld:** `../data/raw/gemeente_dataset.csv`

In [21]:
df = pd.read_csv("../../data/raw/CBS_Kerncijfers.csv", sep=";", encoding="latin-1")
df.head()

,ï»¿ID,Peildatum,WijkenEnBuurten,Gemeentenaam_1,SoortRegio_2,Codering_3,IndelingswijzigingGemeenteWijkBuurt_4,AantalInwoners_5,Mannen_6,Vrouwen_7,...,PersonenPerSoortUitkeringAO_81,PersonenPerSoortUitkeringWW_82,PersonenPerSoortUitkeringAOW_83,JongerenMetJeugdzorgInNatura_84,PercentageJongerenMetJeugdzorg_85,WmoClienten_86,WmoClientenRelatief_87,TotaalDiefstalUitWoningSchuurED_88,VernielingMisdrijfTegenOpenbareOrde_89,GeweldsEnSeksueleMisdrijven_90
0,384,1-1-2025,GM0034,Almere,Gemeente,GM0034,1.0,229574,113738,115836,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,385,1-1-2025,WK003401,Almere,Wijk,WK003401,1.0,1810,880,930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,386,1-1-2025,BU00340101,Almere,Buurt,BU00340101,1.0,795,415,385,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,387,1-1-2025,BU00340102,Almere,Buurt,BU00340102,1.0,1010,465,545,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,388,1-1-2025,WK003402,Almere,Wijk,WK003402,1.0,2890,1395,1500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3) Kolommen checken
Zo zie je hoe de gemeente-naam en codering kolom precies heten.

### 4) Alleen Almere filteren
Filter op de kolom met gemeentenaam (meestal `Gemeentenaam` of `Gemeentenaam_1`).

### 5) Alleen Wijken selecteren (WK-codes)
Filter op de coderingkolom (meestal `Codering` of `Codering_3`).

### 6) Controleren hoeveel wijken overblijven
We tellen unieke wijkcodes en wijknamen.

### 7) Opslaan (optioneel)
Sla de gefilterde dataset op zodat anderen hiermee verder kunnen.
Let op: overleg met team of jullie dit in `/data/processed/` bewaren.

## Data Understanding 
**Doel:** Groei en ontwikkeling van 15–25 jaar per wijk bekijken.

### 1) Kolommen kiezen
Je hebt minimaal nodig:
- Jaar/Peildatum
- Wijknaam
- `k_15Tot25Jaar`

### 2) Jaar netjes maken (als het een datum is)
Als `Peildatum` zoals `1-1-2025` is, pakken we alleen het jaar.

### 3) Groei per wijk (verschil t.o.v. vorig jaar)
We sorteren per wijk en jaar en berekenen verschil en % groei.

### 4) Top wijken met grootste groei (laatste jaar)
Dit helpt om snel inzicht te geven aan de gemeente.

## Data Preparation (Persoon 2)
**Doel:** Ratio jongeren / totaal bevolking maken.

Formule:
- `ratio_0_15 = k_0Tot15Jaar / AantalInwoners`
- `ratio_15_25 = k_15Tot25Jaar / AantalInwoners`

## Modeling & Metrics (Persoon 2)
**Doel:** MAE, RMSE en R² berekenen.

Dit deel gebeurt nadat Persoon 1 het model heeft gebouwd en er een `y_pred` is.
Je hebt nodig:
- `y_test` (echte waarden)
- `y_pred` (voorspelde waarden)

## Korte tekst voor in het verslag 
- Wat zie je bij de groei 15–25?
- Zijn er wijken die sterk stijgen/dalen?
- Wat betekenen de metrics (MAE/RMSE/R²) in normale taal?